In [16]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# https://selenium-python-zh.readthedocs.io/en/latest/index.html selenium中文文档
# https://pandas.pydata.org/pandas-docs/stable/reference/index.html pandas官方文档
# https://www.osgeo.cn/numpy/reference/index.html numpy官方文档
# https://www.runoob.com/python3/python3-tutorial.html python3指南

"""
python的安装
vscode的设置
conda的使用
github/git 的使用
python的基本语法/数据结构/函数/类/模块/包
vscode 中 jupyter notebook的使用
pip的使用
selenium、pandas的使用

"""

'\npython的安装\nvscode的设置\nconda的使用\ngithub/git 的使用\npython的基本语法/数据结构/函数/类/模块/包\nvscode 中 jupyter notebook的使用\npip的使用\nselenium、pandas的使用\n\n'

In [11]:
# 检查下一页是否存在
def check(url:str):
    current_url = browser.current_url
    current_num = int(re.search(r'_\d+',current_url).group().strip('_'))
    next_num = int(re.search(r'_\d+',url).group().strip('_'))
    if current_num < next_num:
        return True
    else:
        return False

# 获取下一页
def next_page():
    bottom = browser.find_elements(By.XPATH,'//div[@class="page"]/span/a')
    url = bottom[-1].get_attribute('href')
    if check(url):
        bottom[-1].click()
    else:
        raise NoSuchElementException('No more pages')

# 获取数据
def get_data(output:list,num = 0)->list:
    elements = browser.find_elements(By.XPATH,'//ul[@class="LifeList clearfix"]/li')
    temp = []
    for i in elements:
        temp.append(i.text.split('\n'))
    if '' in temp:
        num += 1
        get_data(output,num)
    return output + temp

# 清洗数据
def get_area(x:str)->str:
    res = re.findall(r'(菜鸟驿站\()(\S+)(\))',x)
    res = res[0][1]
    return res

# list转化为pd.DataFrame
def transform_pd(data:list)->pd.DataFrame:
    df = pd.DataFrame(data,columns=['名称','区域','地址'])
    df['地址'].replace(r'地址：',r'',regex=True,inplace=True)
    df['名称'] = df['名称'].apply(lambda x: re.findall(r'(菜鸟驿站\()(\S+)(\))',x)[0][1])
    return df

In [12]:
browser = webdriver.Edge()
browser.implicitly_wait(10) # 隐式等待10s
browser.get('http://www.iecity.com/shanghai/brand/75655_1.html')
result = []
while True:
    try:
        result = get_data(result)
        next_page()
    except IndexError:
        continue
    except:
        break
df = transform_pd(result)

In [21]:
# 请求图像链接并使用二进制写入
import requests
response = requests.get('https://avatars.githubusercontent.com/u/35416777?s=40&v=4')
with open(f'image.jpg', 'wb') as f:
    f.write(response.content)

In [8]:
class spider():

    def __init__(self,url:str):
        self.browser = webdriver.Edge()
        self.browser.get(url)

    def check(self,url:str):
        browser = self.browser
        current_url = browser.current_url
        current_num = int(re.search(r'_\d+',current_url).group().strip('_'))
        next_num = int(re.search(r'_\d+',url).group().strip('_'))
        if current_num < next_num:
            return True
        else:
            return False

    def next_page(self):
        browser = self.browser
        bottom = browser.find_elements(By.XPATH,'//div[@class="page"]/span/a')
        url = bottom[-1].get_attribute('href')
        if self.check(url):
            bottom[-1].click()
        else:
            raise Exception('No more pages')

    def get_data(self,output:list,num = 0)->list:
        browser = self.browser
        elements = browser.find_elements(By.XPATH,'//ul[@class="LifeList clearfix"]/li')
        temp = []
        for i in elements:
            temp.append(i.text.split('\n'))
        if '' in temp:
            num += 1
            get_data(output,num)
        return output + temp

    def get_area(x:str)->str:
        res = re.findall(r'(菜鸟驿站\()(\S+)(\))',x)
        res = res[0][1]
        return res

    def transform_pd(data:list)->pd.DataFrame:
        df = pd.DataFrame(data,columns=['名称','区域','地址'])
        df['地址'].replace(r'地址：',r'',regex=True,inplace=True)
        df['名称'] = df['名称'].apply(lambda x: re.findall(r'(菜鸟驿站\()(\S+)(\))',x)[0][1])
        return df
    
    def close(self):
        self.browser.close()

    def get_current_url(self):
        return self.browser.current_url

In [16]:
browser = spider('http://www.iecity.com/shanghai/brand/75655_1.html')
result = []
while True:
    try:
        result = browser.get_data(result)
        browser.next_page()
    except:
        break